In [ ]:
import os, sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt, seaborn as sn
%matplotlib inline
sn.set(rc={'figure.figsize':(11.7,8.27)})

import warnings

warnings.filterwarnings("ignore")

In [ ]:
data_folder = "/home/goubetcle/Documents/Pôle/Challenge 2022/data_challenge/data"


In [ ]:
df_list_station = pd.read_csv(os.path.join(data_folder,"liste_stations.csv"), sep=";", header=0)
df_list_station.head()

In [ ]:
df_prev_sans_obs2020 = pd.read_feather(os.path.join(data_folder, "df_prev_sans_obs2020.feather"))
df_prev_sans_obs2020.head(20)

In [ ]:
df_grille_zoneclimat_fin18 = pd.read_feather(os.path.join(data_folder, "grille_zone_climatique_fin2018.feather"))
df_grille_zoneclimat_fin18.head()

In [ ]:
df_meteo_zone_eol = pd.read_feather(os.path.join(data_folder, "meteo_zone_echeance12_2016_2020_HRES_piEOL_smooth.feather"))
df_meteo_zone_eol.head()

In [ ]:
df_meteo_zone_pv = pd.read_feather(os.path.join(data_folder, "meteo_zone_echeance12_2016_2020_HRES_piPV_smooth.feather"))
df_meteo_zone_pv.head()

In [ ]:
df_prodpv_fc_q90 = pd.read_feather(os.path.join(data_folder, "productionPV_FC_cielclair_q90.feather"))
df_prodpv_fc_q90.head()

In [ ]:
np.unique(df_prev_sans_obs2020.echeance.values)

import pneumathis

client = pneumathis.init()

table_target = "client.data.PUBLIC.SMETEO.PREVISIONS_NEBULOSITE_TEMPERATURE"

sql_query = "SELECT * FROM PUBLIC.SMETEO.PREVISIONS_NEBULOSITE_TEMPERATURE where echeance >= 0 and echeance <=4 and an_utc >= 2016 and an_utc <= 2020"
df_temp = client.query(sql_query)

df_temp.reset_index(drop=True).to_feather(os.path.join(data_folder, "prev_meteo_neb.feather"))

In [ ]:
df_temp = pd.read_feather(os.path.join(data_folder, "prev_meteo_neb.feather"))

df_temp.head()

In [ ]:
np.unique(df_temp.echeance.values)

In [ ]:
def make_zone_columns(df):
    new_df = df.pivot_table(index=['timestamp_utc_str', "echeance"],columns='code_station')
    new_df.columns = [col[-1].strip() for col in new_df.columns.values]
    return new_df



In [ ]:
df_stations = pd.read_csv(os.path.join(data_folder,"WeatherStationsRTE.csv"), header=0, sep=";")
df_stations.head()

In [ ]:
cols_tokeep = ["code_station", "echeance", "prevision_neb", "prevision_temp", "timestamp_utc_str"]

list(make_zone_columns(df_temp[cols_tokeep].astype({"prevision_temp" : float})).index)


In [ ]:
cols_tokeep = ["code_station", "echeance", "prevision_neb", "prevision_temp", "timestamp_utc_str"]

df_meteo_fr = pd.DataFrame()

time_vec, echeance_vec = zip(*list(make_zone_columns(df_temp[cols_tokeep].astype({"prevision_temp" : float})).index))
df_meteo_fr["utc_datetime"] = time_vec
df_meteo_fr["echeance"] = echeance_vec

for col in ["prevision_neb", "prevision_temp"]:
    cols_tokeep = ["code_station", "timestamp_utc_str", "echeance"] + [col]
    newdf = make_zone_columns(df_temp[cols_tokeep].astype({col : float}))
    weights = np.array([float(df_stations[["Poids"]][df_stations.ID == int(c) - 7000].values) for c in newdf.columns])
    df_meteo_fr[col] = np.sum(newdf.values * weights.reshape(1,-1),axis=1)
 
df_meteo_fr.to_csv(os.path.join(data_folder,"meteo_fr.csv"), sep= ";", index=False)
df_meteo_fr.head()